# Fit Multiple Observations

This tutorial shows how to model sources from images observed in different ways, which could mean images taken with the same instrument but different pointings and PSFs, or with different instruments. For this guide we will use a multi-band observation from the Hyper-Suprime Cam (HSC) and a single high-resolution image from the Hubble Space Telescope (HST).

In [ ]:
import astropy.io.fits as fits
import astropy.units as u
# Import Packages
import jax.numpy as jnp
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS

import scarlet2 as sc2

sc2.set_validation(False)

## Load Data

We first load the HSC and HST images, PSFs and weight/variance maps. We also load a catalog of sources detected jointly from the observations (see [here](https://github.com/astro-data-lab/scarlet-test-data/blob/main/scarlet_test_data/data/multiresolution_tutorial/get_source_catalog.py) for details on how this catalog was created).

In [ ]:
from huggingface_hub import hf_hub_download

filename = hf_hub_download(
    repo_id="astro-data-lab/scarlet-test-data",
    filename="multiresolution_tutorial/data.fits.gz",
    repo_type="dataset",
)

with fits.open(filename) as hdul:
    # Load HSC observation
    data_hsc = jnp.array(hdul["HSC_OBS"].data, jnp.float32)
    wcs_hsc = WCS(hdul["HSC_OBS"].header)

    # Load HSC PSF and weights
    psf_hsc_data = jnp.array(hdul["HSC_PSF"].data, jnp.float32)
    obs_hsc_weights = jnp.array(hdul["HSC_WEIGHTS"].data, jnp.float32)

    # Load HST observation
    data_hst = jnp.array(hdul["HST_OBS"].data, jnp.float32)
    wcs_hst = WCS(hdul["HST_OBS"].header)

    # Load HST PSF and weights
    psf_hst_data = jnp.array(hdul["HST_PSF"].data, jnp.float32)
    obs_hst_weights = jnp.array(hdul["HST_WEIGHTS"].data, jnp.float32)

    # Load catalog table and metadata
    coords_table = Table(hdul["CATALOG"].data)
    radecsys = hdul["CATALOG"].header["RADECSYS"]
    equinox = hdul["CATALOG"].header["EQUINOX"]

# Write sources coordinates in SkyCoord
ra_dec = SkyCoord(
    ra=coords_table["RA"] * u.deg,
    dec=coords_table["DEC"] * u.deg,
    frame=radecsys.lower(),
    equinox=f"J{equinox}",
)

Now we create scarlet2 observations and display their contents:

In [ ]:
# Scarlet Observations
obs_hst = sc2.Observation(
    data_hst,
    wcs=wcs_hst,
    psf=psf_hst_data,
    channels=["F814W"],
    weights=obs_hst_weights
)

obs_hsc = sc2.Observation(
    data_hsc,
    wcs=wcs_hsc,
    psf=psf_hsc_data,
    channels=["g", "r", "i", "z", "y"],
    weights=obs_hsc_weights,
)

norm_hst = sc2.plot.AsinhAutomaticNorm(obs_hst)
norm_hsc = (sc2.plot.AsinhAutomaticNorm(obs_hsc))

sc2.plot.observation(
    obs_hst, norm=norm_hst, sky_coords=ra_dec, show_psf=True, label_kwargs={"color": "red"}
);
sc2.plot.observation(obs_hsc, norm=norm_hsc, sky_coords=ra_dec, show_psf=True);

## Create Frame and Observations

We have two different instruments with different pixel resolutions, so we define a model frame that allows us to unify these data:

In [ ]:
model_frame = sc2.Frame.from_observations(
    observations=[obs_hst, obs_hsc],
    coverage="union",  # or "intersection"
)

In addition to defining a common footprint for both observations, the model frame adopts the HST resolution because the HST image has the highest resolution. The channels of both observations are simply concatenated, which means the bands are treated independently across observations even though the wavelengths may overlap.

## Initialize sources from multiple observations

The initialization of the sources follows our general recommendation (from e.g. the [quickstart guide](../0-quickstart)), with the only difference that multiple observations are passed to the `init` functions:

In [ ]:
with sc2.Scene(model_frame) as scene:
    for i, center in enumerate(ra_dec):
        try:
            spectrum, morph = sc2.init.from_gaussian_moments([obs_hst, obs_hsc], center, min_corr=0.99)
        except ValueError:
            spectrum = sc2.init.pixel_spectrum([obs_hst, obs_hsc], center)
            morph = sc2.init.compact_morphology()
        sc2.Source(center, spectrum, morph)

## Fit Multiple Observations

The definition of parameters is completely independent of observations, so nothing changes here:

In [ ]:
from numpyro.distributions import constraints
from functools import partial

spec_step = partial(sc2.relative_step, factor=0.1)
morph_step = partial(sc2.relative_step, factor=1e-3)

with sc2.Parameters(scene) as parameters:
    for i in range(len(scene.sources)):
        sc2.Parameter(
            scene.sources[i].spectrum,
            name=f"spectrum.{i}",
            constraint=constraints.positive,
            stepsize=spec_step,
        )
        sc2.Parameter(
            scene.sources[i].morphology,
            name=f"morph.{i}",
            constraint=constraints.unit_interval,
            stepsize=morph_step,
        )

We could now simply call:

```python
scene.fit([obs_hst, obs_hsc], parameters, max_iter=1000, progress_bar=True)
```

And this would work, but it would take quite a lof of time. The rendering operator for the HSC observation has to change resolution. This operation is much slower than the single-observation than, say, a simple convolution. We therefore resample the HSC observation to the pixel grid of the model frame (which in itself has inherited its pixel grid from the HST observation):

In [ ]:
obs_hsc_ = sc2.CorrelatedObservation.from_observation(obs_hsc, resample_to_frame=model_frame)

norm_hsc_ = sc2.plot.AsinhAutomaticNorm(obs_hsc_)
sc2.plot.observation(obs_hsc_, norm=norm_hsc_, sky_coords=ra_dec, show_psf=True);

The new observation does not require resampling during the fit. And because the resampling correlates the pixel values, we automatically treat it as a {py:class}`~scarlet2.CorrelatedObservation`.

For good measure, we should also account for the pixel correlations in the HST data, which arose because the HST single exposures were also resampled when the HST coadd was created (see the [tutorial](correlated) on correlated data for more details). Since the HST and the model frame share the same pixel grid, we don't need to resample to the model frame here:

In [ ]:
obs_hst_ = sc2.CorrelatedObservation.from_observation(obs_hst)

We can now pass both observations to the fitting method:

In [ ]:
scene_ = scene.fit([obs_hst_, obs_hsc_], parameters, max_iter=1000, progress_bar=True)

Let's check the results:

In [ ]:
sc2.plot.scene(
    scene_,
    observation=obs_hst_,
    show_rendered=True,
    show_observed=True,
    show_residual=True,
    add_labels=True,
    add_boxes=True,
    norm=norm_hst,
    box_kwargs={"edgecolor": "red", "facecolor": "none"},
    label_kwargs={"color": "red"},
)
sc2.plot.scene(
    scene_,
    observation=obs_hsc_,
    show_rendered=True,
    show_observed=True,
    show_residual=True,
    add_labels=True,
    add_boxes=True,
    norm=norm_hsc_,
);

The fit is quite good, but issues remain:

* The model is still quite noisy in the outskirts. A morphology [prior](priors) should help.
* The boxes are too small, so some flux, e.g. around source 10, is not modelled. Better detection footprints would help the initialization of sources.
* There are minor dipoles in the residuals (best seen for source 10), which implies that the astrometry of the HSC and HST data is slightly inconsistent. A fix for this issue is forthcoming.

But the important question is: As we have fit the resampled HSC observation, does the model actually describe the original HSC data well?

In [ ]:
sc2.plot.scene(
    scene_,
    observation=obs_hsc,
    show_rendered=True,
    show_observed=True,
    show_residual=True,
    add_labels=True,
    add_boxes=True,
    norm=norm_hsc,
);

Yeah, it does! The same dipole issue remains and because of the lower resolution, it is now the prevalent feature in the residuals.